## 腾讯算法大赛
1. 历史曝光日志数据文件:
        广告请求 id
        广告请求时间 (timestamp)
        广告位 id
        用户 id
        曝光广告 id
        曝光广告素材尺寸
        曝光广告出价 bid
        曝光广告 pctr -- 预估点击率（统一放大处理）
        曝光广告quality_ecpm
        曝光广告totalEcpm = pctr*bid + quality_ecpm
        
bid 已知
*pctr 预估*
quality_ecpm
2. 用户特征属性文件:  （均为映射值）
        用户 id  
        年龄
        性别
        地域
        婚恋状态
        学历
        消费能力
        设备
        工作状态
        连接类型
        行为兴趣
3. 广告静态数据:
        广告 id
        创建时间
        广告账户 id
        商品 id
        商品类型
        广告行业 id
        素材尺寸
4. 广告操作数据:
        广告 id
        创建/修改时间
        操作类型
        修改字段
        操作后的字段值
      

In [2]:
#import 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

In [2]:
# ### sample training data ###
# # open file
# fin = open('/Users/haoxiran/data_science/腾讯算法大赛/dataset/testA/totalExposureLog.out')
# fout = open("totalExposureLog_sample", "w")
# lines= ""
# # extract n lines data
# for i in range(10000):
#     line = fin.readline()
# #     line = line.replace('\t', ',').replace(' ', ',')
#     lines += line
# fout.write(lines)
# fout.close()
# fin.close()

In [3]:
# count = len(open('/Users/haoxiran/data_science/腾讯算法大赛/dataset/testA/ad_operation.dat', 'r').readlines())
# ad_operation.dat  行数：760866
# count = len(open('/Users/haoxiran/data_science/腾讯算法大赛/dataset/testA/ad_static_feature.out', 'r').readlines())
# ad_static_feature.out  行数：735911
# count = len(open('/Users/haoxiran/data_science/腾讯算法大赛/dataset/testA/user_data', 'r').readlines())
# user_data  行数：1396718
# count = len(open('/Users/haoxiran/data_science/腾讯算法大赛/dataset/testA/totalExposureLog.out', 'r').readlines())
# totalExposureLog.out  行数：102386695
# count = len(open('/Users/haoxiran/data_science/腾讯算法大赛/dataset/testA/test_sample.dat', 'r').readlines())
# test_sample.dat  行数：20290

In [4]:
# import lightgbm as lg

# 数据分析

1. 读入所有数据

In [1]:
# user_data
user_data_feature = ['用户id', '年龄', '性别', '地域', '婚恋状况', '学历', '消费能力', '设备', '工作状态', '连接类型', '行为兴趣']
user_data_df = pd.read_csv('/Users/haoxiran/data_science/腾讯算法大赛/dataset/testA/user_data',sep='\t', names=user_data_feature)
# save as hdf/csv
# user_data_df[['用户id', '年龄', '性别', '地域', '婚恋状况', '学历', '消费能力', '设备', '工作状态', '连接类型']].to_hdf('user1.h5',key ='user1', mode ='w')

# user_data_df['行为兴趣'].to_csv(index=False)
# re-read
# user_data_df = pd.read_hdf('user1.h5')
# user_data_df['行为兴趣'] = pd.read_csv('user2.csv')

# clean version
# user_data_df = pd.read_hdf('user1_clean1.h5')
# user_data_df['行为兴趣'] = pd.read_csv('user2_clean1.csv')



In [ ]:
# totalExposureLog.out
total_exposure_log_feature = ['广告请求id', '广告请求时间', '广告位id', '用户id', '曝光广告id', '曝光广告素材尺寸', '曝光广告出价bid', '曝光广告pctr', '曝光广告quality_ecpm', '曝光广告totalEcpm']
total_exposure_log_df = pd.read_csv('/Users/haoxiran/data_science/腾讯算法大赛/dataset/testA/totalExposureLog.out',sep='\t', names=total_exposure_log_feature)
# save as hdf
# total_exposure_log_df.to_hdf('ad_exposure.h5',key ='ad_exposure', mode ='w')

# re-read
# total_exposure_log_df = pd.read_hdf('ad_exposure.h5')

# clean version
# total_exposure_log_df = pd.read_hdf('ad_exposure_clean1.h5')



In [ ]:
# ad_static_feature
ad_static_feature = ['广告id', '创建时间', '广告账户id', '商品id', '商品类型', '广告行业id', '素材尺寸']
ad_static_feature_df = pd.read_csv('/Users/haoxiran/data_science/腾讯算法大赛/dataset/testA/ad_static_feature.out',sep='\t', names=ad_static_feature)
## preprocess data
size = ad_static_feature_df['素材尺寸'].dropna()
## 
# save as hdf
# ad_static_feature_df.to_hdf('ad_static.h5',key ='ad_static', mode ='w')

# re-read
# ad_static_feature_df = pd.read_hdf('ad_static.h5')

# clean version
# ad_static_feature_df = pd.read_hdf('ad_static_clean1.h5')



In [ ]:
# ad_operation.dat
ad_operation_feature = ['广告id', '创建/修改时间', '操作类型', '修改字段', '操作后的字段值']
ad_operation_df = pd.read_csv('/Users/haoxiran/data_science/腾讯算法大赛/dataset/testA/ad_operation.dat',sep='\t', names=ad_operation_feature)
# save as hdf
# ad_operation_df.to_hdf('ad_operation.h5',key ='ad_operation', mode ='w')

# re-read
# ad_operation_df = pd.read_hdf('ad_operation.h5')

# clean version
# ad_operation_df = pd.read_hdf('ad_operation_clean1.h5')



In [ ]:
# test_sample.dat
test_sample_feature = ['样本id', '广告id', '创建时间', '素材尺寸', '广告行业id', '商品类型', '商品id', '广告账户id', '投放时段', '人群定向', '出价']
test_sample_df = pd.read_csv('/Users/haoxiran/data_science/腾讯算法大赛/dataset/testA/test_sample.dat',sep='\t', names=test_sample_feature)

2. 理解user_data中的用户属性

In [ ]:
## explore the kinds in each specific column

# 探究用户属性中婚恋状况的种类数
# 婚恋状况的种类数： 19

stat = user_data_df.groupby('婚恋状况').describe()
stat_index = stat.index

In [ ]:
stat_set = set('')
# stat_index[0]
# stat_set
for i in stat_index:
    stat_set = set(i.split(',')) | stat_set
# len(stat_set) => 19

*事实证明，用numpy.unique()明显好于python的set集合进行种类划分*

In [ ]:
# 探究用户属性中地域的种类数
# 地域种类数: 13646 

area = user_data_df['地域']
# area_set= set('')
area_list = []
for i in area:
    area_list += i.split(',')
# len(area_list)
area_arr= np.array(area_list)
area_unique = np.unique(area_arr)
# len(area_unique)

In [ ]:
# 探究用户属性中年龄的种类
# 年龄种类数：92

age = user_data_df['年龄']
age_list= []
for i in age:
    age_list += [i]

# age_list
age_arr= np.array(age_list)
age_unique = np.unique(age_arr)
# len(age_unique)

In [ ]:
# 探究用户属性中性别的种类
# 性别种类数：3 
# 1-未知 
# 2、3 -男、女

gender = user_data_df['性别']
gender_list= []
for i in gender:
    gender_list += [i]

# gender_list
gender_arr= np.array(gender_list)
gender_unique = np.unique(gender_arr)
# len(age_unique)

In [ ]:
# 探究用户属性中学历的种类
# 学历种类数：8
# count:
# 1	22990
# 2	337743
# 3	114473
# 4	6352
# 5	349100
# 6	258055
# 7	150252
# 8	157753

edu = user_data_df['学历']
edu_list= []
for i in edu:
    edu_list += [i]

# edu_list
edu_arr= np.array(edu_list)
edu_unique = np.unique(edu_arr)
# len(edu_unique)

In [ ]:
# 探究用户属性中消费能力的种类
# 消费能力种类数：8

consuption_ability = user_data_df['消费能力']
user_data_df.groupby(consuption_ability).count()

In [ ]:
# 探究用户属性中工作的种类
# 工作种类数：7 （0表示未知）

work = user_data_df['工作状态']
# area_set= set('')
work_list = []
for i in work:
    work_list += i.split(',')
# len(area_list)
work_arr= np.array(work_list)
work_unique = np.unique(work_arr)
len(work_unique)

In [ ]:
# 探究用户属性中连接类型的种类
# 连接类型种类数：5 （存在一种未知种类）

consuption_ability = user_data_df['连接类型']
user_data_df.groupby(consuption_ability).count()

In [ ]:
# 探究用户属性中设备的种类
# 设备种类数：3 （0表示未知）

device = user_data_df['设备']
user_data_df.groupby(device).count()

In [ ]:
# 探究用户属性中行为兴趣的种类
# 行为兴趣种类数：2060 （0表示未知） 28 ~ 42429
########## 表达策略：我们要用到one-hot实现对行为兴趣的表达 （42430维矩阵）


## 这里由于数据量巨大，我们采用分区的做法实现分类

behavior = user_data_df['行为兴趣']
# area_set= set('')
final_behavior_unique = np.array([])
for n in range(14):
    for i in behavior[ n*100000 : (n+1)*100000]:
        behavior_list = []
        behavior_list += i.split(',')
        # len(area_list)
        behavior_arr= np.array(behavior_list)
        behavior_unique = np.unique(behavior_arr)
    #
    print("%d epoch finished" % n)
    final_behavior_unique = np.append(final_behavior_unique, behavior_unique)

final_behavior_unique = np.unique(final_behavior_unique)

# len(final_behavior_unique)

3. 理解ad_static中的广告属性

In [ ]:
# 探究广告静态属性中广告id的数量
# 735911为总条数，意味每行均代表不同ad，广告id为唯一标识

id = ad_static_feature_df['广告id']
id_list= []
for i in id:
    id_list += [i]

# gender_list
id_arr= np.array(id_list)
id_unique = np.unique(id_arr)
# len(id_unique)

In [ ]:
# 探究广告静态属性中素材尺寸的种类
# 素材尺寸的种类：65 (其中0为NaN)  max:66    num_max:64   除去0后的总量：509253     空值：2万多
########## solution todo: 将最大的几个值（64-160820, 44-46285, 40-52427, 36-78167, 30-44216）作为被赋值对象赋值到空值中,
##########                数量按比例分配，通过shuffle实现
#                         64:0.4; 44:0.11; 40:0.13; 36: 0.25; 30:0.11
# 目前先直接drop掉含空值行

### 这里存在几处问题： 
### 1.存在NaN值，需填入0
### 2.存在float值，需转化为int
### 3.存在str值，需分解并化为int


# size = ad_static_feature_df['素材尺寸'].fillna(0)
size = ad_static_feature_df['素材尺寸'].dropna()
# area_set= set('')
size_list = []
for i in size:
#     if ',' in i:
#         size_list += i.split(',')
#     else:
#         size_list += [i]
#     inted = int(i)
#     size_list += [inted]
    if type(i) == str:
        if ',' in i:
            print(i)
            size_list += i.split(',')
        else:
            size_list += [int(i)]
    elif type(i) == float:
        size_list += [int(i)]
    elif type(i) == int:
        size_list += [i]

# size_arr= np.array(size_list)
# size_unique = np.unique(size_arr)
# len(size_unique)

## 
# size_arr= np.array(size_list)
# size_arr = size_arr.astype(int)
# np.argmax(np.bincount(size_arr))
# np.argwhere(np.bincount(size_arr) > 40000) 

4. 理解ad_operation中的广告属性

5. 理解test_sample(create_time) / total_exposure_log(request_time) / ad_static(create_time) / ad_operation(modify_time)的时间性质

In [ ]:
# 将曝光日志中的请求时间转为日期并排序 (这里的时间我们可以考虑北京时间作为我们的时间)
total_exposure_log_df['广告请求时间'] = pd.to_datetime(total_exposure_log_df['广告请求时间'], unit='s')
total_exposure_log_df['广告请求时间']= total_exposure_log_df.广告请求时间.dt.month*100 + total_exposure_log_df.广告请求时间.dt.day
total_exposure_log_df = total_exposure_log_df.sort_values('广告请求时间')
# total_exposure_log_df.to_hdf('ad_exposure_clean2.h5', key= 'ad_exposure_clean',mode='w')

# 以'曝光广告id','广告请求时间', '曝光广告素材尺寸' 为主键
tmp_df = total_exposure_log_df.groupby(['曝光广告id','广告请求时间'], as_index=False).count()
tmp2_df = total_exposure_log_df.groupby(['曝光广告id','广告请求时间'], as_index=False).mean()

# 构建训练集
train_feature = ['广告id','日期', '曝光广告素材尺寸', '曝光广告出价bid',  '日曝光量']
train_df = pd.DataFrame(columns=train_feature)

# 
# train_df[['广告id', '日期', '曝光广告素材尺寸', '日曝光量']] = tmp_df[['曝光广告id', '广告请求时间', 
#                                                        '曝光广告素材尺寸', '曝光广告出价bid']]
# train_df['曝光广告出价bid']  = tmp2_df['曝光广告出价bid']


In [ ]:
# 探究广告曝光日志中广告请求时间的范围
#
# total_exposure_log 广告请求时间: 2019-02-16 22:59:27 (1550329167) ~~ 2019-03-19 23:58:44 (1553011124)
####### NOTE: 这里我们要获取整个的时间作为我们的训练集中数据

# test_sample 创建时间： 2015-06-04 17:11:32（1433409092） ~~~  2019-03-24 15:52:33（1553413953） 
# ad_static 创建时间:  2014-06-03 22:31:20(1401805880) ~~ 2019-03-25 21:49:28 (1553521768)
# ad_operation 修改时间： 2019/2/16 ~ 2019/2/30(除29日) ; 3/1 ~ 3/19

##### 注：这里我们需要清洗掉ad_operation的2月30日的脏数据

# total_exposure_log
time = total_exposure_log_df['广告请求时间']
time_list = []
for i in time:
    time_list += [int(i)]

time_arr= np.array(time_list)
time_unique = np.unique(time_arr)
len(time_unique)
# sorted_time_list = time_unique.tolist()
# sorted_time_list.sort()
# sorted_time_list[0]
# sorted_time_list[-1]



In [ ]:
# test_sample
#
time = test_sample_df['创建时间']
time_list = []
for i in time:
    time_list += [int(i)]

time_arr= np.array(time_list)
time_unique = np.unique(time_arr)
len(time_unique)
# np.sort(time_unique)

In [ ]:
# ad_static
#
time = ad_static_feature_df['创建时间']
time_list = []
for i in time:
    time_list += [int(i)]

time_arr= np.array(time_list)
time_unique = np.unique(time_arr)
len(time_unique)
# np.sort(time_unique)

In [ ]:
# ad_operation (不同于timestamp，这里采用的是datetime, 这里的date大多是省去时分秒的，这个时间主要用于时间的排序)
##### 注意：这里每一天的timestamp都可能有多个值，这里通过np.bincount()得到了, 发现这里只有33天为主要的时间
#  2/16 ~ 2/30(除29日) ; 3/1 ~ 3/19    -- 包含两端
#  这里我们尽量不要省掉时分秒时间，每一天均有数万的值（最高达到近3万）
#  因此，如果我们使用onehot encoding,我们可以设一个30000维的向量代表修改时间（包括日期或星期，以及时分秒）

#  另外：2月只到28日，其中的30号这一天有待讨论
#  对于0值。我们不需讨论

time = ad_operation_df['创建/修改时间']
time_list = []
for i in time:
    
    time_list += [str(i)]

time_arr= np.array(time_list)
time_unique = np.unique(time_arr)
len(time_unique)

# np.bincount(subtime_list)
# np.argwhere(np.bincount(subtime_list) > 0)

In [ ]:
# 探究广告创建时间的范围 ( 2015-04-07 09:43:55+08:00 -- 2019-03-19 13:24:10+08:00)


6. 理解广告静态数据、广告操作数据和曝光日志中广告id的联系

In [ ]:
##### 这里，我们以ad_static作为基准，在ad_operation和ad_exposure中只保留这一部分的广告。

# 因为，如果有ad_static，没有ad_operation，广告的基本属性依然存在，可以存在没有操作的情况
# 但是，如果只有ad_operation，则无法利用此广告id作为训练集（即我们始终要保证每条广告的基本属性）

# totalExposureLog.out
# id总数

id = total_exposure_log_df['曝光广告id']
id_list = []
for i in id:
    id_list += [int(i)]

id_arr= np.array(id_list)
id_unique = np.unique(id_arr)
# len(id_unique)


In [ ]:
# ad_static
# id总数

id = ad_static_feature_df['广告id']
id_list = []
for i in id:
    id_list += [int(i)]

id_arr= np.array(id_list)
id_unique = np.unique(id_arr)
len(id_unique)

In [ ]:
# ad_operation
# id总数

id = ad_operation_df['广告id']
id_list = []
for i in id:
    id_list += [int(i)]

id_arr= np.array(id_list)
id_unique = np.unique(id_arr)
len(id_unique)

In [ ]:
# 广告特征作为一个重复出现的值，是具有分类意义的，故广告id不可舍去
id = test_df['广告id']
id_list = []
for i in id:
    id_list += [int(i)]

id_arr= np.array(id_list)
id_unique = np.unique(id_arr)
# len(id_unique) => 1900


# 数据预处理

7. 数据清洗

#### 对于ad_static的清洗

In [ ]:
# 对于ad_static的清洗 -- 清洗后的总数据（735911 -> 497665）

# 去除创建时间为0的行
# 去除广告行业存在多值的行

# ad_static_feature_df = pd.read_hdf('ad_static.h5')
drop_list=set('')
for i in range(len(ad_static_feature_df)):
    r = ad_static_feature_df.iloc[i]
    if (type(r['商品id']) == str):
        if (r['商品id'].find(',') > -1):
            drop_list.add(i)
    if (type(r['广告行业id']) == str):
        if r['广告行业id'].find(',') > -1:
            drop_list.add(i)
    if (type(r['素材尺寸']) == str):
        if r['素材尺寸'].find(',') > -1:
            drop_list.add(i)
    if(r['创建时间'] == 0):
        drop_list.add(i)
drop_list = list(drop_list)
# drop elements
ad_static_feature_df = ad_static_feature_df.drop(drop_list)

# 去除素材尺寸空值
ad_static_feature_df = ad_static_feature_df.dropna()

# 将商品id转成int值
# 将广告行业id转成int
# 将素材尺寸转成int
ad_static_feature_df['商品id'] = ad_static_feature_df['商品id'].astype('int64')
ad_static_feature_df['广告行业id'] = ad_static_feature_df['广告行业id'].astype('int64')
ad_static_feature_df['素材尺寸'] = ad_static_feature_df['素材尺寸'].astype('int64')

# ad_static_feature_df.info()

## reset index
ad_static_feature_df = ad_static_feature_df.reset_index()
ad_static_feature_df = ad_static_feature_df.drop(['index'], axis=1)

# ad_static_feature_df.to_hdf('ad_static_clean1.h5',key ='ad_static_clean', mode ='w')

#### 对ad_operation的清洗

In [ ]:
# 对ad_operation的清洗 -- 清洗后的数据（760866 -> 614377）

# 清洗掉ad_static中没有的广告id（以ad_static为基准）
# 因为，如果有ad_static，没有ad_operation，广告的基本属性依然存在，可以没有操作
# 但是，如果只有ad_operation，则无法利用此广告id作为训练集（即我们始终要保证每条广告的基本属性）

static_id = set(ad_static_feature_df['广告id'].tolist())
operation_id = set(ad_operation_df['广告id'].tolist())
# 获取ad_static中没有的广告id
# drop_id = set('')
for i in operation_id:
    if i not in static_id:
        drop_id.add(i)
drop_id = list(drop_id)

# 删除需要drop的广告id的行
ad_operation_df.drop(ad_operation_df['广告id'].loc[drop_id].index)

# reset index
ad_operation_df = ad_operation_df.reset_index()
ad_operation_df = ad_operation_df.drop(['index'], axis=1)

# ad_operation_df.to_hdf('ad_operation_clean1.h5', key= 'ad_operation_clean',mode='w')

In [ ]:
# 清洗掉请求时间为2/30的异常数据

# 清除2月30日的数据
drop_list=[]
for i in range(len(ad_operation_df)):
    r = ad_operation_df.iloc[i]
    if str(r['创建/修改时间'])[4:8] == '0230':
        drop_list += [i]

ad_operation_df = ad_operation_df.drop(drop_list)

# reset index
ad_operation_df = ad_operation_df.reset_index()
ad_operation_df = ad_operation_df.drop(['index'], axis=1)

# ad_operation_df.to_hdf('ad_operation_clean1.h5', key= 'ad_operation_clean',mode='w')

#### 对ad_exposure的清洗

In [ ]:
# 对ad_exposure的清洗
# 

# 清洗含字段缺失的数据
total_exposure_log_df = total_exposure_log_df.dropna()

# 清洗ad_static中不存在的广告id数据
drop_id = set('')

static_id = set(ad_static_feature_df['广告id'].tolist())
exposure_id = set(total_exposure_log_df['曝光广告id'].tolist())

drop_id = exposure_id - (static_id & exposure_id)
# useless id to NaN
def is_in_set(x):
    if x in drop_id:
        return np.nan
    else:
        return x
total_exposure_log_df['曝光广告id'] = total_exposure_log_df['曝光广告id'].apply(lambda x: is_in_set(x))
# drop NaN id
total_exposure_log_df = total_exposure_log_df.dropna()

# id return to int
total_exposure_log_df['曝光广告id'] = total_exposure_log_df['曝光广告id'].astype('int64')
# reset index
total_exposure_log_df = total_exposure_log_df.reset_index()
total_exposure_log_df = total_exposure_log_df.drop(['index'], axis=1)

# total_exposure_log_df.to_hdf('ad_exposure_clean1.h5', key= 'ad_exposure_clean',mode='w')

In [ ]:
# 去重（同一用户，同一广告id，同一请求时间，同一请求id，同一广告位id，同一素材尺寸的记录视为用户操作不当）
total_exposure_log_df = total_exposure_log_df.drop_duplicates(['广告请求id', '广告请求时间', '广告位id', '用户id', '曝光广告id', '曝光广告素材尺寸'])

# total_exposure_log_df.to_hdf('ad_exposure_clean1.h5', key= 'ad_exposure_clean',mode='w')



In [ ]:
# 去除时间为非整天的数据(即216)





#### 对user的清洗

In [ ]:
# 对user_data的清洗(无用user清洗)
#### 由于user id只在历史曝光日志中出现，且用户id应该一一对应，因此，我们需要除掉
#### 在这里，len(user_id & exposure_id)=1341958；len(user_id)=1396718；len(exposure_id)=1341958
#### 因此，我们只需除去user里面的多余用户即可

drop_id = set('')

user_id = set(user_data_df['用户id'].tolist())
exposure_id = set(total_exposure_log_df['用户id'].tolist())

drop_id = user_id - (user_id & exposure_id)

# useless id to NaN
def is_in_set(x):
    if x in drop_id:
        return np.nan
    else:
        return x
user_data_df['用户id'] = user_data_df['用户id'].apply(lambda x: is_in_set(x))

user_data_df = user_data_df.dropna()
# id return to int
user_data_df['用户id'] = user_data_df['用户id'].astype('int64')
# reset index
user_data_df = user_data_df.reset_index()
user_data_df = user_data_df.drop(['index'], axis=1)

# user_data_df[['用户id', '年龄', '性别', '地域', '婚恋状况', '学历', '消费能力', '设备', '工作状态', '连接类型']].to_hdf('user1_clean1.h5',key ='user1_clean', mode ='w')
# user_data_df['行为兴趣'].to_csv('user2_clean1.csv', index=False)

8. 对测试集中特征值的分析

*这里广告id、商品类型、素材尺寸、广告行业id、创建时间、广告账户id这几个特征与ad_static相关联，因此为固定特征不需讨论<br>*
*投放时段、人群定向、出价这几个特征中，出价与ad_exposure有关，投放时段与人群定向和ad_operation有关<br>*
*这里呢，我们需要将投放时段和人群定向作为训练集特征，因此我们训练集中广告id为所有在ad_operation中存在的id*

9. 训练集及其对应的测试集的构建
-----

在这里我们必定需要考虑到曝光日志和广告操作日志的问题，但是我们可以先简化问题：<br>
（1） 原始特征：
* 只考虑曝光日志：利用广告id的单一性计算每天的曝光量（其中主键为曝光广告id、日期和尺寸），出价则使用平均值
* 细致考虑曝光日志：我们还需要考虑到素材尺寸变化和出价的问题，主键再添加有尺寸和出价


* 对数据测试的重新实现：我们先从训练集中获取5%的数据作为我们的线下测试集，然后再进行交叉验证，最后我们将我们的模型重新对整个训练集进行训练

-----
* 初步思路：计算每个广告id在一个自然日内的日曝光量  52%

In [ ]:
# 将曝光日志中的请求时间转为日期并排序 (这里的时间我们可以考虑北京时间作为我们的时间)
total_exposure_log_df['广告请求时间'] = pd.to_datetime(total_exposure_log_df['广告请求时间'], unit='s')
total_exposure_log_df['广告请求时间']= total_exposure_log_df.广告请求时间.dt.month*100 + total_exposure_log_df.广告请求时间.dt.day


####### 清洗掉2月16日的数据（非整天）
total_exposure_log_df = total_exposure_log_df[total_exposure_log_df.广告请求时间 >216]
# total_exposure_log_df = total_exposure_log_df.sort_values('广告请求时间')
# total_exposure_log_df.to_hdf('ad_exposure_clean2.h5', key= 'ad_exposure_clean',mode='w')

# 以'曝光广告id','广告请求时间', '曝光广告素材尺寸' 为主键
tmp_df = total_exposure_log_df.groupby(['曝光广告id','广告请求时间'], as_index=False).count()
tmp2_df = total_exposure_log_df.groupby(['曝光广告id','广告请求时间'], as_index=False).mean()

# 构建训练集
train_feature = ['广告id','日期', '曝光广告素材尺寸', '曝光广告出价bid',  '日曝光量']
train_df = pd.DataFrame(columns=train_feature)

# 
train_df[['广告id', '日期', '日曝光量']] = tmp_df[['曝光广告id', '广告请求时间', '曝光广告出价bid']]
train_df[['曝光广告出价bid', '曝光广告素材尺寸']]  = tmp2_df[['曝光广告出价bid', '曝光广告素材尺寸']]

train_df['曝光广告素材尺寸'] = train_df['曝光广告素材尺寸'].astype('int64')
# train_df.to_hdf('train_data1.h5', key='train1', mode='w')
# re-read
# train_df = pd.read_hdf('train_data1.h5')


#### 为什么我们在这里用到了素材尺寸作为我们的主键：

#### 已知'曝光广告id','广告请求时间'是我们的主属性，那么我们对于尺寸进行了判断，发现其在加入主键前后对数据集行数影响不大，则可以将其作为主键
#### 至于出价我发现它在每个广告的不同时段均有不同的价格，换句话说，它在随人为或自动的变化，这个便不能作为主键，而是进行平均值处理放入训练集



In [ ]:
# 测试集（样本id，出价，素材尺寸，日曝光量）
test_feature = ['广告id', '曝光广告素材尺寸', '曝光广告出价bid',  '预测日曝光量']
test_df = pd.DataFrame(columns=test_feature)

test_df[['广告id', '曝光广告素材尺寸', '曝光广告出价bid']] = test_sample_df[['广告id', '素材尺寸', '出价']]

# test_df.to_hdf('test_data1.h5', key='test1', mode='w')
# re-read
# test_df = pd.read_hdf('test_data1.h5')

* 进一步思路：尺寸作为变量并不是很好，所以将其取出重新计算频数 50%

In [ ]:
# 将曝光日志中的请求时间转为日期并排序 (这里的时间我们可以考虑北京时间作为我们的时间)
total_exposure_log_df['广告请求时间'] = pd.to_datetime(total_exposure_log_df['广告请求时间'], unit='s')
total_exposure_log_df['广告请求时间']= total_exposure_log_df.广告请求时间.dt.weekday
total_exposure_log_df = total_exposure_log_df.sort_values('广告请求时间')
# total_exposure_log_df.to_hdf('ad_exposure_clean2.h5', key= 'ad_exposure_clean',mode='w')

# 以'曝光广告id','广告请求时间', '曝光广告素材尺寸' 为主键
tmp_df = total_exposure_log_df.groupby(['曝光广告id','广告请求时间'], as_index=False).count()
tmp2_df = total_exposure_log_df.groupby(['曝光广告id','广告请求时间'], as_index=False).mean()

# 构建训练集
train_feature = ['广告id','星期', '曝光广告素材尺寸', '曝光广告出价bid',  '日曝光量']
train_df = pd.DataFrame(columns=train_feature)

# 
train_df[['广告id', '星期', '日曝光量']] = tmp_df[['曝光广告id', '广告请求时间', '曝光广告出价bid']]
train_df[['曝光广告出价bid', '曝光广告素材尺寸']]  = tmp2_df[['曝光广告出价bid', '曝光广告素材尺寸']]
train_df['曝光广告素材尺寸'] = train_df['曝光广告素材尺寸'].astype('int64')
# train_df.to_hdf('train_data1.h5', key='train1', mode='w')
# re-read
# train_df = pd.read_hdf('train_data1.h5')

###### 总结：发现测试集中不存在日期这一特征，故将其去掉，但是可以考虑

* 扩大特征范围：'广告id', '创建时间', '素材尺寸', '广告行业id', '商品类型', '商品id', '广告账户id', '出价'，'投放时间', '人群定向'

#### 我们将训练集分为验证集和训练集进而帮助我们测试（计划按1/10 或 1/20 的数量作为验证集）

In [ ]:
ad_exposure['广告请求时间'] = pd.to_datetime(ad_exposure['广告请求时间'], unit='s')

date = pd.to_datetime(ad_exposure['广告请求时间'], unit='s')
ts = pd.Series(np.random.randn(len(date)),index=date)
# type(ts.tz_localize('Asia/Shanghai').index)
datetime = ts.tz_localize('Asia/Shanghai').index
ad_exposure['广告请求时间'] = datetime

ad_exposure['广告请求时间']= ad_exposure.广告请求时间.dt.month*100 + ad_exposure.广告请求时间.dt.day
ad_exposure = ad_exposure.sort_values('广告请求时间')
# total_exposure_log_df.to_hdf('ad_exposure_clean2.h5', key= 'ad_exposure_clean',mode='w')

ad_exposure = ad_exposure[ad_exposure.广告请求时间>216]
# ad_exposure.to_hdf('ad_exposure_clean2.h5', key= 'ad_exposure_clean2',mode='w')

tmp_df = ad_exposure.groupby(['曝光广告id','广告请求时间'], as_index=False).count()
tmp2_df = ad_exposure.groupby(['曝光广告id','广告请求时间'], as_index=False).mean()

train_feature = ['广告id','日期', '曝光广告素材尺寸', '曝光广告出价bid',  '日曝光量']
train_df = pd.DataFrame(columns=train_feature)

# 
train_df[['广告id', '日期', '日曝光量']] = tmp_df[['曝光广告id', '广告请求时间', '曝光广告出价bid']]
train_df[['曝光广告出价bid', '曝光广告素材尺寸']]  = tmp2_df[['曝光广告出价bid', '曝光广告素材尺寸']]
train_df['曝光广告出价bid'] = train_df['曝光广告出价bid'].round(2)
train_df['曝光广告素材尺寸'] = train_df['曝光广告素材尺寸'].astype('int64')

tmp = pd.merge(train_df, ad_static, on=['广告id'])
# tmp.drop(['素材尺寸', '日期'],axis =1)
tmp = tmp.drop(['素材尺寸', '日期'],axis =1)
train_df = tmp

# 测试集处理时间特征
date = pd.to_datetime(train_df['创建时间'], unit='s')
ts = pd.Series(np.random.randn(len(date)),index=date)
# type(ts.tz_localize('Asia/Shanghai').index)
datetime = ts.tz_localize('Asia/Shanghai').index
train_df['创建时间'] = datetime
train_df['创建星期'] = train_df['创建时间'].dt.weekday

# train_df.to_hdf('train_data2.h5', key= 'train2',mode='w')

In [ ]:
# 加入特征：投放时间、人群定向

# 载入ad-operation； 通过id、时间和修改字段进行分组并筛选出其中的投放时间和人群定向（规定单日多字条时，人群定向自动选择最长的字符串，投放时间选择第一个）
# 对投放时间的共性处理：取第一个为当前请求的投放时间，且将空缺值用最近天数的投放时间属性来填充
# 对人群定向的共性处理：取在同一天内字符串最长的为当天人群定向属性，且将空缺值用最近天数的投放时间属性来填充


# 加入面向人群
tmp_period = pd.read_hdf('period_tmp.h5')
tmp_people = pd.read_hdf('people_tmp.h5')
train_df = pd.read_hdf('train_data3.h5')

tmp_people = tmp_people.rename(columns={'peopleModifyDate':'日期'})
tmp_merge = pd.merge(train_df, tmp_people, on=['广告id'])
# 共有id
ad_id = tmp_merge['广告id'].drop_duplicates()
tmp_merge = pd.merge(train_df, ad_id, on=['广告id'])
tmp_merge = pd.merge(train_df, tmp_people, how='outer', on=['广告id', '日期'])

tmp_merge = tmp_merge.sort_values('广告id').sort_values('日期')
# 填充存在面向人群特征却为空的数据（即为在当天之前已经设置了特征属性）
tmp_merge['面向人群'] = tmp_merge['面向人群'].fillna(method = 'ffill')
tmp_merge = tmp_merge.dropna()

tmp_merge = tmp_merge.reset_index(drop=True)
tmp_merge['投放时段'] = tmp_merge['投放时段'].astype('int64')

# tmp_merge.to_hdf('train_data3.h5', key='train_data3', mode='w')


In [ ]:
# 切分训练集（按1/20切分）
train_df[train_df.日期 == 319].to_hdf('train_val_data.h5', key='train_val', mode='w')
train_df[train_df.日期 != 319].to_hdf('train_tri_data.h5', key='train_tri', mode='w')


In [ ]:
# 构建测试集
test_df = test_sample_df[['广告id', '创建时间', '投放时间', '素材尺寸', '广告行业id', '商品类型', '商品id', '广告账户id', '出价']]

# 处理投放时段特征
test_df['投放时间'] = test_df['投放时间'].apply(lambda x: x.split(',')[0])

# 测试集处理时间特征
date = pd.to_datetime(test_df['创建时间'], unit='s')
ts = pd.Series(np.random.randn(len(date)),index=date)
# type(ts.tz_localize('Asia/Shanghai').index)
datetime = ts.tz_localize('Asia/Shanghai').index
test_df['创建时间'] = datetime


# test_df.to_hdf('test_data2.h5', key='test2', mode='w')

10. 评估标准的理解和分析

* 对最终得分的分析：

这里通过提交成绩我们得知单调性规则的最高值为60分左右，因此 w2 * (mono_score + 1) /2的最大值为0.6<br>

另外，我们知道，mono_score的最大值为1，则w2 * (mono_score + 1) /2最大值为w2,亦即w2 = 0.6<br>

并且，要达到最佳效果，sample值要最小，即为0，则通过解答得w1 = 0.4

In [ ]:
# totalScore公式
total_score= 0.4 * (1- sample/2) + 0.6 * (mono_score + 1) /2

* 对准确性指标SAMPLE的分析和理解：<br>
**注意：这里的参数不可直接使用(均为array类型)**

In [ ]:
sample = sum(2 * abs(predict - label)/(predict + label)) / predict.size

* 对出价单调相关性指标的分析：

In [ ]:

test_sampledf=pd.merge(test_sampledf,test_sampledf_resultdf, how='left', on='广告id')
test_sampledf.sort_values(by=["广告id","出价"],inplace=True)
    
# 作为基准
standard=test_sampledf.groupby(by='广告id').head(1)
standard.rename(columns={'样本id':'基准样本id','出价':'基准出价','曝光量':'基准曝光量'},inplace=True)
    
test_sampledf=pd.merge(test_sampledf,standard,how="left",left_on='广告id', right_on='广告id')
test_sampledf['score']=test_sampledf.apply(
        lambda x: (
                  ((x['基准曝光量']-x['曝光量'])* (x['基准出价']-x['出价']))/
                   abs((x['基准曝光量']-x['曝光量'])* (x['基准出价']-x['出价']))
                )
    ,axis=1
)
    
monoscore=test_sampledf.groupby(by='广告id')['score'].mean().mean()

In [ ]:
train_tri_label_df = pd.read_hdf('train_tri_label_data.h5')

train_tri_result_df = train_tri_label_df.join(pd.Series(result, name='result'))

def getScore()



11. 分析测试集中广告id的有关信息 （4.30）

------
* 测试集中广告id和广告静态属性集中的id的讨论:<br>

    测试集中广告id的类别：1954 <br>
    测试集总记录数：20290<br>
    测试集和ad-static集的共有广告id的类别：1330<br>
    
由此可知，测试集并不和ad-static重合，存在一部分广告没有被记录在ad-static中，
因此，我们只能使用测试集中的特征来实现我们的。

另外，我们也可以知道，每一条记录为某广告在一定价格时的预估日曝光量。
这样一来，我们可以尝试将出价和其他原始特征分开来分析。

In [ ]:
test_df = pd.read_hdf('test_data1.h5')
ad_static_feature_df = pd.read_hdf('ad_static_clean1.h5')

# ad_static
# id总数

id = test_df['广告id']
id_list = []
for i in id:
    id_list += [int(i)]

id_arr= np.array(id_list)
id_unique = np.unique(id_arr)
len(id_unique)

static_id = set(ad_static_feature_df['广告id'].tolist())
exposure_id = set(test_df['广告id'].tolist())
drop_id = exposure_id - (static_id & exposure_id)

# len(drop_id)
# len(static_id & exposure_id)

* 对出价单调相关性指标的分析：

In [ ]:
def getMonoScore (samplefilename,submissionfilename):
    test_samplefile = pd.read_csv(samplefilename,sep='\t',
                              nrows=30000,header=None,names=['样本id','广告id','创建时间','素材尺寸','广告行业id'
                                                           ,'商品类型','商品id','广告账户id','投放时间','人群定向'
                                                            ,'出价'],usecols=['样本id','广告id','出价'])
    test_sampledf = pd.DataFrame(test_samplefile)
    
    test_sampledf_result=pd.read_csv(submissionfilename,
                                  sep=",",header=None,names=["样本id","曝光量"])
    test_sampledf_resultdf= pd.DataFrame(test_sampledf_result)
    
    test_sampledf=pd.merge(test_sampledf,test_sampledf_resultdf, how='left', left_on='样本id', right_on='样本id')
    test_sampledf.sort_values(by=["广告id","出价"],inplace=True)
    
    # 作为基准
    standard=test_sampledf.groupby(by='广告id').head(1)
    standard.rename(columns={'样本id':'基准样本id','出价':'基准出价','曝光量':'基准曝光量'},inplace=True)
    
    test_sampledf=pd.merge(test_sampledf,standard,how="left",left_on='广告id', right_on='广告id')
    test_sampledf['score']=test_sampledf.apply(
        lambda x: (
                  ((x['基准曝光量']-x['曝光量'])* (x['基准出价']-x['出价']))/
                   abs((x['基准曝光量']-x['曝光量'])* (x['基准出价']-x['出价']))
                  )
        ,axis=1
    )
    
    monoscore=test_sampledf.groupby(by='广告id')['score'].mean().mean()
    print("经过相关性计算成绩为："+str(monoscore))
    print("预估相关性部分成绩为："+ str(60*(monoscore+1)/2))

# 特征构造

12. 原始特征: **投放时段**、**人群定向**、**出价**、广告id、商品类型、素材尺寸、广告行业id、创建时间、广告账户id、商品id

In [ ]:
# 归一化处理

from sklearn.preprocessing import MinMaxScaler
X= train_df['曝光广告素材尺寸'].values.reshape(-1, 1)
size_scaler = MinMaxScaler().fit(X)
X = size_scaler.transform(X)
# size_scaler.scale_  
train_df['曝光广告素材尺寸_scaled'] = X

X= train_df['曝光广告出价bid'].values.reshape(-1, 1)
bid_scaler = MinMaxScaler().fit(X)
X = bid_scaler.transform(X)
# bid_scaler.scale_  
train_df['曝光广告出价bid_scale'] = X

13. 稀疏特征（one-hot）

* 对**投放时段**进行独热处理

In [ ]:
# 转换为binary并填充至64位
train_df['bin'] = train_df['投放时段'].apply(lambda x: bin(x)[2:])
train_df['bin'] = train_df['bin'].apply(lambda x: x.zfill(48))

# str to array
bin_list = list(train_df['bin'])
bin_list = [list(x) for x in bin_list]
bin_list = [[int(y) for y in x] for x in bin_list]
bin_arr = np.array(bin_list)
bin_df = pd.DataFrame(bin_arr)

train_df.join(bin_df)
train_df = train_df.reset_index(drop=True)
train_df.drop(['bin','投放时段'], axis=1, inplace = True)

for i in range(48):
    train_df.rename(columns={i:'时段'+str(i)}, inplace=True)

# int64 to int32 for memory saving
train_df[train_df.columns.difference(['曝光广告出价bid', '创建时间', '面向人群'])]=train_df[train_df.columns.difference(['曝光广告出价bid', '创建时间', '面向人群'])].astype('int32')

# train_df.to_hdf('train_sparse_data1.h5', key='train_sparse_data1', mode='w')
# train_df.info()

* 对**广告账户id**进行独热处理

In [ ]:
id = pd.get_dummies(train_df['广告账户id'])
id.index.name='账户id'

train_df = train_df.sort_values(['广告id', '日期'])
id.to_hdf('train_sparse_acc_id.h5', key='train_sparse_acc_id', mode='w')

* 对**创建星期**进行独热处理

In [ ]:
train_df = train_df.join(pd.get_dummies(train_df['创建星期']), lsuffix='创建星期')
for i in range(7):
    if i in train_df.columns:
        train_df.rename(columns={i:'创建星期'+str(i)}, inplace=True)

train_df.drop(['创建星期'],axis=1, inplace=True)

* 对**广告行业id**进行独热处理

In [ ]:
train_df = train_df.join(pd.get_dummies(train_df['广告行业id']), lsuffix='广告行业id')
for i in range(1, 252):
    if str(i) in train_df.columns:
        train_df.rename(columns={str(i):'行业'+str(i)}, inplace=True)
        
train_df.drop(['广告行业id', '-1'],axis=1, inplace=True)

* 对**创建时间**进行独热处理

In [ ]:
train_df['创建时间'] = (train_df['创建时间'].dt.year-2000) * 10000 + (train_df['创建时间'].dt.month * 100) + train_df['创建时间'].dt.day

train_df = train_df.join(pd.get_dummies(train_df['创建时间']))

for i in range(150406, 190319):
    if i in train_df.columns.values:
        train_df.rename(columns={i:'创建时间'+str(i)}, inplace=True)
train_df.rename(columns={190319:'创建时间'+str(190319)}, inplace=True)

train_df.drop('创建时间',axis=1, inplace=True)


* 对**商品类型**进行独热处理

In [ ]:
train_df = train_df.join(pd.get_dummies(train_df['商品类型']))
for i in range(20):
    train_df.rename(columns={i:'商品类型'+str(i)}, inplace=True)

train_df.drop(['商品类型'],axis=1, inplace=True)

* 对**商品id**进行独热处理

In [ ]:
id = pd.get_dummies(train_df['商品id'])
id.index.name='商品id'

id.to_hdf('train_sparse_product_id.h5', key='train_sparse_product_id', mode='w')

14. 向量长度特征

15. XGB、LGB组合特征

16. 长度特征